In [1]:
import os
import numpy as np
import pandas as pd
from mediapipe import solutions

from utils import smooth_data
from video_converter import Video2DataFrame
from custom_pose_landmarks import CustomPoseLandmark

c:\Users\nemet\DeepLearning\sc_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Prepare paths
data_path = '../data/'
video_path = os.path.join(data_path, 'raw/squat')

# Read personal data from excel
personal_data = pd.read_excel(os.path.join(data_path, 'PersonalData.xlsx'))

# Processed data path
processed_data = os.path.join(data_path, 'processed')

In [20]:
# Selected values of pose landmarks corresponding to PoseLandmark class from MediaPipe library
values = [0, 11, 12, 13, 14, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28, 31, 32]

# Custom pose landmark names and their connections
landmarks = {
    'THORAX': ['NOSE'],
    'PELVIS': ['LEFT_HIP', 'RIGHT_HIP'],}

# MediaPipe solutions
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

custom_pose = CustomPoseLandmark(mp_pose, values, landmarks)
conv = Video2DataFrame(mp_pose, mp_drawing, custom_pose)

In [21]:
converter = Video2DataFrame(mp_pose, mp_drawing, custom_pose)

# Convert videos to dataframe
dataframe = converter.get_dataframe(
    source=video_path,
    detection=0.9,
    tracking=0.9,
    video_display=False
)

Converting 001_01_05_01_040_1_C file to dataframe...


c:\Users\nemet\DeepLearning\StrengthCoach\notebooks\video_converter.py:188: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat(


In [ ]:
# Save dataframe
# dataframe.to_csv('ConvertedSquats.csv', index=False)

# Load dataframe
dataframe = pd.read_csv(os.path.join(processed_data, 'ConvertedSquats.csv'))

In [ ]:
# Canonical form code
default_hip_width = 1
default_pelvis = np.zeros(shape=(3,))

transformed_data = pd.DataFrame()

for _, file_data in dataframe.groupby(by='FileId'):
    # Extract starting pose coordinates
    starting_pose = file_data.iloc[0]

    # Z-axis normalization procedure
    left_hip = starting_pose.filter(regex='LeftHip').to_numpy()
    right_hip = starting_pose.filter(regex='RightHip').to_numpy()

    left_knee = starting_pose.filter(regex='LeftKnee').to_numpy()
    right_knee = starting_pose.filter(regex='RightKnee').to_numpy()

    left_thigh_length = np.linalg.norm(left_hip - left_knee)
    right_thigh_length = np.linalg.norm(right_hip - right_knee)

    mean_length = np.mean([left_thigh_length, right_thigh_length])

    # Normalization
    to_normalize = file_data.filter(regex='Z$')
    # min_z = to_normalize.min()
    # max_z = to_normalize.max()

    # normalized = 2 * mean_length * (to_normalize - min_z) / (max_z - min_z) - mean_length
    normalized = mean_length * to_normalize

    file_data = file_data.assign(**normalized)


    # Extract starting pose coordinates
    starting_pose = file_data.iloc[0]
    
    # Translation procedure
    pelvis = starting_pose.filter(regex='Pelvis').to_numpy()

    translation_vector = default_pelvis - pelvis

    # Scaling procedure
    left_hip = starting_pose.filter(regex='LeftHip').to_numpy()
    right_hip = starting_pose.filter(regex='RightHip').to_numpy()

    hip_vector = left_hip - right_hip
    hip_width = np.linalg.norm(hip_vector)

    scale_factor = default_hip_width / hip_width

    # Transformation
    to_transform = file_data.filter(regex='X$|Y$|Z$')

    transformed = scale_factor * (
        to_transform + np.tile(translation_vector, len(to_transform.columns) // 3)
    )

    file_data = file_data.assign(**transformed)
    transformed_data = pd.concat([transformed_data, file_data])

dataframe = transformed_data.copy()

In [ ]:
# Merge personal and video data
data = pd.merge(dataframe, personal_data, on='Id')

# Calculate the maximum load that was passed
max_load = data.loc[data['Lifted'] == 1, ['Id', 'Load']].groupby(by='Id', as_index=False).max()
max_load = max_load.rename(columns={'Load': 'MaxLoad'})
data = pd.merge(data, max_load, on='Id')

# Calculate what percentage of the maximum load is the current load
data['PercentageMaxLoad'] = 100 * data['Load'] / data['MaxLoad']

del data['MaxLoad']

# Get only lifted approaches
data = data.loc[data['Lifted'] == 1]

# Variables that aren't needed in the first run
to_drop = [
    'Id', 'Age', 'Height', 'Weight', 'PastInjuries', 'LastInjury', 'PainDuringTraining', 'SquatRecord',
    'BenchPressRecord', 'DeadliftRecord', 'PhysicalActivities', 'SetNumber', 'Load', 'Lifted', 'Timestamp']

data = data.drop(columns=to_drop)

# Categorical variables that need to be one hot encoded
to_one_hot = [
    'ProficiencyLevel', 'EquipmentAvailability', 'TrainingProgram', 'TrainingFrequency', 'CameraPosition']

data = pd.get_dummies(data, columns=to_one_hot, dtype=int)

# Move the PercentageMaxLoad column to the end of the dataframe
percentage = data.pop('PercentageMaxLoad')
data['PercentageMaxLoad'] = percentage

# Smooth all features extracted from MediaPipe solution
data = smooth_data(data, frac=0.1, it=3)

In [ ]:
data

In [3]:
# Save dataframe
# data.to_csv(os.path.join(processed_data, 'OneRepMaxData_250324.csv'), index=False)

# Load dataframe
data = pd.read_csv(os.path.join(processed_data, 'OneRepMaxData_240324.csv'))